# Importing Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Loading Datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # Datasets are fetched from google drive(previously stored)

Mounted at /content/drive


In [ ]:
df1 = pd.read_csv("/content/drive/My Drive/Fraud Detection/dataset1_train.csv")
df2 = pd.read_csv("/content/drive/My Drive/Fraud Detection/dataset2_train.csv")
df3 = pd.read_csv("/content/drive/My Drive/Fraud Detection/dataset3_train.csv")

# Data Preprocessing

In [ ]:
df1.info()

In [ ]:
df2.info()

In [ ]:
df3.info()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# Split each dataset into train and test
X1_train, X1_test, y1_train, y1_test = train_test_split(df1.drop(['class'],axis=1), df1['class'], test_size=0.2, stratify=df1['class'], random_state=42)
X2_train, X2_test, y2_train, y2_test = train_test_split(df2.drop(['Class'],axis=1), df2['Class'], test_size=0.2, stratify=df2['Class'], random_state=42)
X3_train, X3_test, y3_train, y3_test = train_test_split(df3.drop(['is_fraud'],axis=1), df3['is_fraud'], test_size=0.2, stratify=df3['is_fraud'], random_state=42)

# Normalize features
scaler1, scaler2, scaler3 = StandardScaler(), StandardScaler(), StandardScaler()
X1_train, X1_test = scaler1.fit_transform(X1_train), scaler1.transform(X1_test)
X2_train, X2_test = scaler2.fit_transform(X2_train), scaler2.transform(X2_test)
X3_train, X3_test = scaler3.fit_transform(X3_train), scaler3.transform(X3_test)

# Convert to PyTorch tensors
X1_train, X1_test = torch.tensor(X1_train, dtype=torch.float32), torch.tensor(X1_test, dtype=torch.float32)
X2_train, X2_test = torch.tensor(X2_train, dtype=torch.float32), torch.tensor(X2_test, dtype=torch.float32)
X3_train, X3_test = torch.tensor(X3_train, dtype=torch.float32), torch.tensor(X3_test, dtype=torch.float32)
y1_train, y1_test = torch.tensor(y1_train.to_numpy(), dtype=torch.float32), torch.tensor(y1_test.to_numpy(), dtype=torch.float32)
y2_train, y2_test = torch.tensor(y2_train.to_numpy(), dtype=torch.float32), torch.tensor(y2_test.to_numpy(), dtype=torch.float32)
y3_train, y3_test = torch.tensor(y3_train.to_numpy(), dtype=torch.float32), torch.tensor(y3_test.to_numpy(), dtype=torch.float32)

# Create PyTorch DataLoaders
batch_size = 32
train_loader1 = DataLoader(TensorDataset(X1_train, y1_train), batch_size=batch_size, shuffle=True)
train_loader2 = DataLoader(TensorDataset(X2_train, y2_train), batch_size=batch_size, shuffle=True)
train_loader3 = DataLoader(TensorDataset(X3_train, y3_train), batch_size=batch_size, shuffle=True)


# Training of Autoencoder and Extracting Encoded Features

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Define a simple autoencoder
class Autoencoder(nn.Module):
    def __init__(self, input_size, encoding_dim=16):
        super(Autoencoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, encoding_dim)  # Compressed representation
        )

        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 64),
            nn.ReLU(),
            nn.Linear(64, input_size)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded


In [ ]:
# Function to train an autoencoder
def train_autoencoder(autoencoder, X, epochs=15, lr=0.001):
    optimizer = optim.Adam(autoencoder.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    for epoch in range(epochs):
        optimizer.zero_grad()
        encoded, decoded = autoencoder(X)
        loss = loss_fn(decoded, X)  # Reconstruction loss
        loss.backward()
        optimizer.step()

    return autoencoder

# Train autoencoders on training sets
autoencoder1 = train_autoencoder(Autoencoder(X1_train.shape[1]), X1_train)
autoencoder2 = train_autoencoder(Autoencoder(X2_train.shape[1]), X2_train)
autoencoder3 = train_autoencoder(Autoencoder(X3_train.shape[1]), X3_train)

# Extract encoded features for train and test sets
X1_train_encoded = autoencoder1.encoder(X1_train)
X1_test_encoded = autoencoder1.encoder(X1_test)

X2_train_encoded = autoencoder2.encoder(X2_train)
X2_test_encoded = autoencoder2.encoder(X2_test)

X3_train_encoded = autoencoder3.encoder(X3_train)
X3_test_encoded = autoencoder3.encoder(X3_test)

In [ ]:
# Combine tensors along the first dimension (rows)
X_test_combined = torch.cat((X1_test_encoded, X2_test_encoded, X3_test_encoded), dim=0)
y_test_combined = torch.cat((y1_test, y2_test, y3_test), dim=0)

In [ ]:
client1_train_loader = DataLoader(TensorDataset(X1_train_encoded, y1_train), batch_size=batch_size, shuffle=True)
client2_train_loader = DataLoader(TensorDataset(X2_train_encoded, y2_train), batch_size=batch_size, shuffle=True)
client3_train_loader = DataLoader(TensorDataset(X3_train_encoded, y3_train), batch_size=batch_size, shuffle=True)

# Model Building

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import copy

# Define the Multi-Input Fraud Detection Model
class FraudDetectionModel(nn.Module):
    def __init__(self, input_dims):  # input_dims = [dim1, dim2, dim3] (Different for each client)
        super(FraudDetectionModel, self).__init__()
        self.num_clients = len(input_dims)

        # Create separate input layers for each client's feature space
        self.input_layers = nn.ModuleList([
            nn.Sequential(
                nn.Linear(input_dim, 32),
                nn.ReLU()
            ) for input_dim in input_dims
        ])

        # Final classifier after aggregation
        self.classifier = nn.Sequential(
            nn.Linear(32 * self.num_clients, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x_list):
        # Process each client's input separately
        processed_features = [self.input_layers[i](x_list[i]) for i in range(self.num_clients)]

        # Concatenate processed features before classification
        x_combined = torch.cat(processed_features, dim=1)
        return self.classifier(x_combined)

# Define each client's input feature dimension
client_feature_dims = [client1_train_loader.dataset.tensors[0].shape[1],
                       client2_train_loader.dataset.tensors[0].shape[1],
                       client3_train_loader.dataset.tensors[0].shape[1]]

server_model = FraudDetectionModel(client_feature_dims)


# Model Training

In [ ]:
# Function to train a model locally on a client's dataset
def train_local_model(model, train_loader, client_idx, epochs=10, lr=0.001):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    for epoch in range(epochs):
        total_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()

            # Ensure tensors are detached to avoid computation graph retention
            X_batch = X_batch.detach()
            y_batch = y_batch.detach()

            # Create zero inputs for other clients
            input_list = [X_batch if i == client_idx else torch.zeros_like(X_batch).detach()
                          for i in range(model.num_clients)]

            y_pred = model(input_list).squeeze()
            loss = criterion(y_pred, y_batch)

            loss.backward(retain_graph=False)  # Ensure the graph is not retained
            optimizer.step()
            total_loss += loss.item()

        print(f"Client {client_idx+1}, Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_loader)}")

    return model.state_dict()


# Function to aggregate weights from multiple clients
def federated_averaging(global_model, local_weights):
    new_state_dict = copy.deepcopy(global_model.state_dict())

    for key in new_state_dict.keys():
        new_state_dict[key] = torch.stack([local_weights[i][key] for i in range(len(local_weights))], dim=0).mean(dim=0)

    global_model.load_state_dict(new_state_dict)
    return global_model

# Federated Learning Training
num_rounds = 3
train_loaders = [client1_train_loader, client2_train_loader, client3_train_loader]

for round_num in range(num_rounds):
    print(f"\n--- Federated Training Round {round_num+1} ---")
    local_weights = []

    for i, train_loader in enumerate(train_loaders):
        print(f"\nTraining on Client {i+1}...")
        local_model = copy.deepcopy(server_model)
        local_weights.append(train_local_model(local_model, train_loader, i))  # Train & store weights

    # Aggregate local weights into global model
    server_model = federated_averaging(server_model, local_weights)
    print("Global Model Updated!")



--- Federated Training Round 1 ---

Training on Client 1...
Client 1, Epoch 1/10, Loss: 0.2607587755688793
Client 1, Epoch 2/10, Loss: 0.23694331872894722
Client 1, Epoch 3/10, Loss: 0.2320861233454523
Client 1, Epoch 4/10, Loss: 0.2299233409046117
Client 1, Epoch 5/10, Loss: 0.22774593726829948
Client 1, Epoch 6/10, Loss: 0.22666801666557915
Client 1, Epoch 7/10, Loss: 0.22558766350745682
Client 1, Epoch 8/10, Loss: 0.22467276880832152
Client 1, Epoch 9/10, Loss: 0.2240871781697265
Client 1, Epoch 10/10, Loss: 0.22358732917550667

Training on Client 2...
Client 2, Epoch 1/10, Loss: 0.010454719487861076
Client 2, Epoch 2/10, Loss: 0.0036688045497278634
Client 2, Epoch 3/10, Loss: 0.003496953032618586
Client 2, Epoch 4/10, Loss: 0.0033765184234126927
Client 2, Epoch 5/10, Loss: 0.003234263747394942
Client 2, Epoch 6/10, Loss: 0.003146052499884174
Client 2, Epoch 7/10, Loss: 0.0031119513966017718
Client 2, Epoch 8/10, Loss: 0.003033942130806493
Client 2, Epoch 9/10, Loss: 0.003042435788

# Saving the Model(optional)

In [ ]:
#torch.save(server_model.state_dict(), "/content/drive/My Drive/Fraud Detection/server_model.pth")

In [ ]:
#server_model.load_state_dict(torch.load("/content/drive/My Drive/Fraud Detection/server_model.pth"))
#server_model.eval()  # Set the model to evaluation mode for inference

<All keys matched successfully>

# Evaluation on Test Data

In [ ]:
test_loader = DataLoader(TensorDataset(X_test_combined, y_test_combined), batch_size=batch_size, shuffle=True)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score, precision_recall_curve, roc_curve, roc_auc_score
import torch

# Function to evaluate the model on a given test dataset
def evaluate(model, test_loaders):
    model.eval()

    # Iterate over each client and evaluate on their test data
    for i, test_loader in enumerate(test_loaders):
        y_true, y_pred_probs = [], []
        #correct, total = 0, 0

        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                # Ensure tensors are detached to avoid computation graph retention
                X_batch = X_batch.detach()
                y_batch = y_batch.detach()

                # Create zero inputs for other clients, similar to the training phase
                input_list = [X_batch if i == client_idx else torch.zeros_like(X_batch).detach()
                              for client_idx in range(model.num_clients)]

                y_pred = model(input_list).squeeze()  # Forward pass
                y_pred_probs.extend(y_pred.cpu().numpy())  # Store predicted probabilities
                y_true.extend(y_batch.cpu().numpy())  # Store actual labels

                # predictions = (y_pred >= 0.5).float()  # Convert predictions to binary values
                # correct += (predictions == y_batch).sum().item()  # Count correct predictions
                # total += y_batch.size(0)

        # Convert to numpy arrays
        y_true = np.array(y_true)
        y_pred_probs = np.array(y_pred_probs)

        # Compute and print accuracy, precision, recall, F1-score, and PR-AUC
        # accuracy = correct / total
        # print(f"Test Accuracy: {accuracy:.4f}")

        # Threshold tuning: find best threshold based on F1-score
        precisions, recalls, thresholds = precision_recall_curve(y_true, y_pred_probs)
        f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-8)
        best_index = np.argmax(f1_scores)
        best_threshold = thresholds[best_index] if best_index < len(thresholds) else 0.5

        # Compute Precision, Recall, F1-score
        y_pred_binary = (y_pred_probs >= best_threshold).astype(int)
        accuracy = (y_pred_binary == y_true).mean()
        precision = precision_score(y_true, y_pred_binary)
        recall = recall_score(y_true, y_pred_binary)
        f1 = f1_score(y_true, y_pred_binary)
        pr_auc = average_precision_score(y_true, y_pred_probs)  # PR-AUC Score
        roc_auc = roc_auc_score(y_true, y_pred_probs)  # ROC-AUC Score

        print(f"Test Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-score: {f1:.4f}")
        print(f"PR-AUC: {pr_auc:.4f}")
        print(f"Roc-AUC score: {roc_auc:.4f}")


test_loaders = [test_loader]

# Evaluate the model
evaluate(server_model, test_loaders)

Test Accuracy: 0.9778
Precision: 0.0557
Recall: 0.0461
F1-score: 0.0504
PR-AUC: 0.0210
Roc-AUC score: 0.6070
